In [ ]:
import pandas as pd#0단계
import time

# --- 1. 경로 설정 ---
CSV_FILE_PATH = 'train.csv'
FEATHER_FILE_PATH ='train_original.feather'

print(f"--- [CSV 변환 시작] '{CSV_FILE_PATH}' 로딩 (약 6분 20초 소요)... ---")
start_time = time.time()

try:
    # 1. 6분 걸리는 CSV 로드 (parse_dates 포함)
    df = pd.read_csv(CSV_FILE_PATH, parse_dates=['time'])
    
    load_time = time.time()
    print(f"--- 로드 완료. (소요 시간: {load_time - start_time:.2f} 초) ---")
    
    # 2. '초고속' Feather 파일로 저장 (핵심)
    print(f"--- [Feather 저장] '{FEATHER_FILE_PATH}' 파일로 저장 중... ---")
    df.to_feather(FEATHER_FILE_PATH)
    
    print(f"--- [변환 완료] (소요 시간: {time.time() - load_time:.2f} 초) ---")
    print(f"\n성공: 이제부터 '{FEATHER_FILE_PATH}' 파일을 사용하면 됩니다.")

except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {CSV_FILE_PATH}")
except Exception as e:
    print(f"[오류] 변환 중 문제가 발생했습니다: {e}")

--- [CSV 변환 시작] 'train.csv' 로딩 (약 6분 20초 소요)... ---
--- 로드 완료. (소요 시간: 511.82 초) ---
--- [Feather 저장] 'train_original.feather' 파일로 저장 중... ---
--- [변환 완료] (소요 시간: 7.48 초) ---

성공: 이제부터 'train_original.feather' 파일을 사용하면 됩니다.


In [ ]:
import pandas as pd#1~3단계
import numpy as np
import time
import os

# --- 1. 경로 설정 ---
# (이전 '스마트' 코드에서 생성된 초고속 파일 경로)
TRAIN_FEATHER_PATH = 'train_original.feather'
TEST_FEATHER_PATH = 'test_original.feather'

# [핵심] 분리할 '정답(Y)' 및 '정답 파생' 컬럼 목록
# 'energy'는 train에만 있고 nins에서 파생되었으므로, '누수'를 막기 위해 반드시 함께 제거
TARGET_COLS_TO_SEPARATE = ['nins', 'energy']

print("--- [ 'One Universe' 파이프라인 1-3단계 시작 ] ---")
start_time = time.time()

try:
    # --- 1단계: 데이터 로드 ---
    print(f"'{TRAIN_FEATHER_PATH}' (A세계) 로딩 중...")
    df_train = pd.read_feather(TRAIN_FEATHER_PATH)
    print(f"'{TEST_FEATHER_PATH}' (B세계) 로딩 중...")
    df_test = pd.read_feather(TEST_FEATHER_PATH)

    print("\n--- [데이터 로드 완료] ---")
    print(f"Train 원본 행: {len(df_train)}")
    print(f"Test 원본 행: {len(df_test)}")

    # --- 2단계: 'Y' 분리 (가장 중요) ---
    print("\n--- [ 2단계: 'Y' (정답) 분리 시작 ] ---")
    
    # 정답(nins) 백업
    if 'nins' in df_train.columns:
        y_train = df_train['nins'].copy()
        print(f"'y_train' (nins) 백업 완료. (총 {len(y_train)} 행)")
    else:
        raise ValueError("[오류] 'nins' 컬럼이 train_original.feather에 없습니다!")

    # Train 데이터에서 '정답' 및 '정답 파생' 컬럼 모두 제거
    cols_to_drop_train = [col for col in TARGET_COLS_TO_SEPARATE if col in df_train.columns]
    df_train = df_train.drop(columns=cols_to_drop_train)
    print(f"Train에서 정답 컬럼 {cols_to_drop_train} 제거 완료.")
    
    # Test 데이터에도 'nins'가 있다면(원본 CSV에는 없었지만 확인차) 제거
    cols_to_drop_test = [col for col in TARGET_COLS_TO_SEPARATE if col in df_test.columns]
    if cols_to_drop_test:
        df_test = df_test.drop(columns=cols_to_drop_test)
        print(f"Test에서 {cols_to_drop_test} 제거 완료.")


    # --- 3단계: 'X' 결합 (A/B 세계 통일) ---
    print("\n--- [ 3단계: 'X' (특징) 결합 시작 ] ---")
    
    # 'type' 컬럼 (train/test)이 이미 있으므로 그대로 합칩니다.
    # 이 'type' 컬럼이 나중에 다시 두 데이터를 분리할 유일한 열쇠입니다.
    df_full = pd.concat([df_train, df_test], ignore_index=True)

    print("--- [ 'One Universe' (df_full) 생성 완료 ] ---")
    
    # --- 결과 검증 ---
    print("\n--- [결과 검증] ---")
    print(f"  통합 데이터(df_full) 총 행: {len(df_full)}")
    print(f"  (예상: {len(df_train) + len(df_test)})")
    print(f"  'nins'가 df_full에 있는가? {'nins' in df_full.columns}")
    print(f"  'type' 컬럼 고유값: {df_full['type'].unique()}")

    end_time = time.time()
    print(f"\n--- [ 1-3단계 완료 ] (총 소요 시간: {end_time - start_time:.2f} 초) ---")
    print("\n이제 'df_full' 변수에 A/B 세계가 통일된 'X 특징' 데이터가 준비되었습니다.")
    print("다음 단계는 이 'df_full'에 '단일' 보간(91.6% NaN 해결)을 적용하는 것입니다.")


except FileNotFoundError:
    print("[오류] 'train_original.feather' 또는 'test_original.feather' 파일을 찾을 수 없습니다.")
    print("이전 '스마트 분석' 코드가 먼저 실행되어 파일이 생성되었는지 확인하세요.")
except Exception as e:
    print(f"[오류] 파이프라인 중 문제가 발생했습니다: {e}")

In [ ]:
import pandas as pd#4단계
import numpy as np
import time

# --- [가정] ---
# 1-3단계가 완료되어 'df_full'과 'y_train' 변수가
# 메모리에 로드되어 있다고 가정합니다.

print("--- [ 'One Universe' 파이프라인 4단계 시작 ] ---")
print("--- '단일' 보간 규칙 (ffill) 적용 ---")
start_time = time.time()

try:
    # --- 4-1: 보간 대상 컬럼 정의 ---
    # 1-3단계에서 'nins', 'energy'는 이미 제거되었습니다.
    # 91.6% NaN이었던 모든 날씨/센서 컬럼 목록입니다.
    COLS_TO_INTERPOLATE = [
        'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 
        'pressure', 'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 
        'temp_b', 'uv_idx', 'vis', 'wind_chill_temp', 'wind_dir_b', 
        'wind_gust_spd', 'wind_spd_b', 'cloud_a', 'ground_press', 
        'humidity', 'rain', 'snow', 'temp_a', 'temp_max', 'temp_min', 
        'wind_dir_a', 'wind_spd_a'
    ]

    # df_full에 실제로 존재하는 컬럼만 필터링 (안전장치)
    cols_exist = [col for col in COLS_TO_INTERPOLATE if col in df_full.columns]
    print(f"총 {len(cols_exist)}개 컬럼에 대해 91.6% NaN 보간을 시작합니다...")

    # --- 4-2: '단일' 보간 실행 (핵심) ---
    
    # 1. 'pv_id'별로 그룹화합니다.
    # 2. 'ffill()' (Forward Fill)을 적용하여 '10:00' 값으로 '10:05'~'10:55'를 채웁니다.
    # 3. 'bfill()' (Backward Fill)을 마지막에 적용하여, 
    #    혹시 모를 '00:00'의 초기 NaN 값을 뒤에서 당겨와 채웁니다.
    
    # [주의] 이 작업은 시간이 다소 걸릴 수 있습니다.
    df_full[cols_exist] = df_full.groupby('pv_id')[cols_exist].ffill().bfill()

    print("\n--- [ '단일' 보간 완료 ] ---")
    
    # --- 4-3: 결과 검증 ---
    print("\n--- [보간 결과 검증] ---")
    
    # 'cloud_a' 컬럼의 NaN 개수를 다시 확인합니다. 0이 되어야 합니다.
    nan_count_cloud_a = df_full['cloud_a'].isna().sum()
    print(f"  'cloud_a'의 남은 NaN 개수: {nan_count_cloud_a}")
    
    if nan_count_cloud_a == 0:
        print("  (성공) 'A/B 세계'의 91.6% NaN 문제가 '단일' 규칙으로 해결되었습니다.")
    else:
        print("  [경고] 보간 후에도 NaN이 남아있습니다. 추가 확인이 필요합니다.")
        
    end_time = time.time()
    print(f"\n--- [ 4단계 완료 ] (총 소요 시간: {end_time - start_time:.2f} 초) ---")
    print("\n이제 'df_full'은 'A/B 세계'가 통일되고, 보간까지 완료된 '진짜' 특징(X) 데이터가 되었습니다.")
    print("다음 단계는 이 'df_full'에 '단일' KMeans 군집(증거3)을 적용하는 것입니다.")


except NameError:
    print("[오류] 'df_full' 변수를 찾을 수 없습니다.")
    print("1-3단계 코드가 먼저 실행되어 'df_full'이 메모리에 있는지 확인하세요.")
except Exception as e:
    print(f"[오류] 4단계 보간 중 문제가 발생했습니다: {e}")

In [1]:
import pandas as pd#5단계
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# --- [가정] ---
# 4단계가 완료되어 'df_full' (보간 완료) 변수가
# 메모리에 로드되어 있다고 가정합니다.

print("--- [ 'One Universe' 파이프라인 5단계 시작 ] ---")
print("--- '단일' Scaler + KMeans 군집 적용 (증거2, 3 해결) ---")
start_time = time.time()

# --- 5-1: 군집에 사용할 피처 정의 ---
# 'A/B 세계' 분석(증거 2)에 따라 좌표('coord1', 'coord2')를 사용합니다.
# (참고: 원본 스크립트가 다른 피처를 썼다면, 여기에 추가해야 합니다.)
CLUSTER_FEATURES = ['coord1', 'coord2']

# KMeans 군집 개수 (보고서에서 20개로 가정)
N_CLUSTERS = 20

try:
    print(f"{N_CLUSTERS}개 군집 생성을 시작합니다...")

    # --- 5-2: '단일' StandardScaler 적용 (증거 2: 타입 충돌 해결) ---
    # 210개 ID 전체 데이터로 '단일' Scaler를 'fit'합니다.
    scaler = StandardScaler()
    
    # [핵심] fit_transform을 df_full 전체에 적용
    # float32/float64 충돌이 원천적으로 발생할 수 없습니다.
    scaled_features = scaler.fit_transform(df_full[CLUSTER_FEATURES])
    print("  '단일' StandardScaler 적용 완료.")

    # --- 5-3: '단일' KMeans 적용 (증거 3: 맵 병합 실패 해결) ---
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
    
    # [핵심] 210개 ID 전체 데이터로 '단일' KMeans를 'fit'합니다.
    # 이렇게 하면 '비어있는 군집'이 발생하지 않습니다.
    cluster_labels = kmeans.fit_predict(scaled_features)
    print("  '단일' KMeans 적용 (fit_predict) 완료.")

    # --- 5-4: 결과 할당 ---
    df_full['cluster_id'] = cluster_labels
    print("  'df_full'에 'cluster_id' 컬럼 생성 완료.")

    # --- 5-5: 결과 검증 ---
    print("\n--- [군집 결과 검증] ---")
    
    # 'cluster_id'가 0~19 사이의 값을 가지는지 확인
    print(f"  생성된 군집 ID 범위: {df_full['cluster_id'].min()} ~ {df_full['cluster_id'].max()}")
    
    # 'pv_id'와 'cluster_id'가 1:1로 잘 매칭되었는지 확인
    # (pv_id별로 cluster_id가 1개인지)
    pv_cluster_map = df_full.drop_duplicates(subset=['pv_id', 'cluster_id'])
    
    print(f"  총 210개 ID가 군집에 할당되었는지 확인: {pv_cluster_map['pv_id'].nunique() == 210}")
    
    
    end_time = time.time()
    print(f"\n--- [ 5단계 완료 ] (총 소요 시간: {end_time - start_time:.2f} 초) ---")
    print("\n이제 'df_full'은 'A/B 세계'의 핵심 오류(보간, 군집)가 모두 해결되었습니다.")
    print("다음 단계는 이 'df_full'을 다시 A(Train)/B(Test)로 분리하는 것입니다.")


except NameError:
    print("[오류] 'df_full' 변수를 찾을 수 없습니다.")
    print("1-4단계 코드가 먼저 실행되었는지 확인하세요.")
except Exception as e:
    print(f"[오류] 5단계 군집 중 문제가 발생했습니다: {e}")

--- [ 'One Universe' 파이프라인 5단계 시작 ] ---
--- '단일' Scaler + KMeans 군집 적용 (증거2, 3 해결) ---
20개 군집 생성을 시작합니다...
[오류] 'df_full' 변수를 찾을 수 없습니다.
1-4단계 코드가 먼저 실행되었는지 확인하세요.
